In [56]:
import arcpy
import os
from arcpy import analysis as AN
from arcpy import management as DM
from arcpy.sa import *
from collections import Counter
env = arcpy.env
env.overwriteOutput = True
env.workspace = os.getcwd() + "\\" "ExpectedAccidents.gdb"

In [57]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})
    
def get_counts(in_features, field):
    return Counter([row[0] for row in arcpy.da.SearchCursor(in_features, field)])

def fct_lump(in_features, field, n):
    counts = get_counts(in_features, field)
    keep = [x[0] for x in counts.most_common(n)]
    with arcpy.da.UpdateCursor(in_features, [field]) as cursor:
        for row in cursor:
            if row[0] not in keep:
                row[0] = "Other"
            cursor.updateRow(row)
    del row, cursor 
    
def SelectAggregateWrite(selectionFeature, polygonFeature, sel, fieldName, outPolygon):
    try:
        selLayer = DM.SelectLayerByAttribute(selectionFeature, "NEW_SELECTION", sel)
        nRecords = DM.GetCount(selLayer)
    except arcpy.ExecuteError as e:
        print(e)
    print(f"{nRecords[0]} selected.")
    tmpLyr = "tmp"
    DM.CopyFeatures(selLayer, tmpLyr)
    # tmpLyr = 'in_memory\\tmp.lyr'
    # DM.MakeFeatureLayer(selLayer, tmpLyr) 
    DM.SelectLayerByAttribute(selectionFeature, "CLEAR_SELECTION")
    AN.SummarizeWithin(polygonFeature, tmpLyr, outPolygon)
    DM.AlterField(outPolygon, 'Point_Count', fieldName, fieldName)
    print(f"{outPolygon} created.")
    DM.Delete(tmpLyr)

In [158]:
print(f"Feature classes: {arcpy.ListFeatureClasses()}")
print(f"Tables: {arcpy.ListTables()}")
print(f"Rasters: {arcpy.ListRasters()}")

Feature classes: ['transbase_collisions_XY', 'SFIntersections_Voronoi_Collisions', 'SFIntersections_Voronoi_Collisions_Join', 'SFIntersections_Voronoi_Clipped', 'broadsides_collisions', 'headons_collisions', 'merged', 'broadsides', 'headons', 'rearends', 'darkness', 'wetness', 'proceeding_straight', 'making_left_turn', 'unsafe_speed', 'unsafe_turn', 'collisions', 'pcf_viol_c_Unsafe_turn_or_lane_change', 'pcf_viol_c_Other', 'pcf_viol_c_Red_signal', 'pcf_viol_c_Unsafe_speed', 'pcf_viol_c_Unsafe_turn_or_lane', 'pcf_viol_c_Violation_of_right_', 'type_of_co_Broadside', 'type_of_co_Head_On', 'type_of_co_Hit_Object', 'type_of_co_Other', 'type_of_co_Rear_End', 'type_of_co_Sideswipe', 'lighting_Dark___Street_Lights', 'lighting_Daylight', 'lighting_Dusk___Dawn', 'lighting_Other', 'road_surfa_Dry', 'road_surfa_Other', 'road_surfa_Wet', 'party1_mov_Changing_Lanes', 'party1_mov_Making_Left_Turn', 'party1_mov_Making_Right_Turn', 'party1_mov_Making_U_Turn', 'party1_mov_Other', 'party1_mov_Proceeding_

# Isolate San Francisco polygon

In [58]:
albers3310 = arcpy.SpatialReference(3310)
DM.Project('BayAreaCounties.shp', 'BayAreaCounties3310.shp', albers3310)
sf_county = DM.SelectLayerByAttribute('BayAreaCounties3310.shp', "NEW_SELECTION", '"county" = \'San Francisco\'')
DM.CopyFeatures(sf_county, "sf_county.shp")
DM.MultipartToSinglepart("sf_county.shp", "sf_county_polygons.shp")

arcpy.AddField_management("sf_county_polygons.shp", "Shape_area", "DOUBLE")
with arcpy.da.UpdateCursor("sf_county_polygons.shp", ["SHAPE@AREA","Shape_area"]) as cursor:
    for row in cursor:
        row[1] = row[0]
        cursor.updateRow(row)
del row, cursor 

areas = []
with arcpy.da.SearchCursor("sf_county_polygons.shp", ["Shape_area"]) as cursor:
    for row in cursor:
        areas.append(row[0])
del row, cursor 
sql = f'"Shape_area" = {max(areas)}'
sf_city = DM.SelectLayerByAttribute("sf_county_polygons.shp", "NEW_SELECTION", sql)
DM.CopyFeatures(sf_city, "sf_city_boundary.shp")
env.extent = arcpy.Describe("sf_city_boundary.shp").extent

# Convert collisions to Points, Project to 3310

In [59]:
DM.XYTableToPoint("transbase_collisions.csv", "collisions", "longitude", "latitude")
DM.Project("collisions", "collisions3310.shp", albers3310)

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\collisions3310.shp'>

# Create Voronoi Tesselation around Intersections

In [60]:
# DM.UnsplitLine("SFSpeedLimits", "streets_unsplit", ["from_st", "to_st"])
# AN.Intersect("streets_unsplit", "intersections", output_type = "POINT")

DM.Project("SFIntersections.shp", "SFIntersections3310.shp", albers3310)
AN.CreateThiessenPolygons("SFIntersections3310", "SFIntersections_Voronoi.shp", "ALL")
AN.Clip("SFIntersections_Voronoi.shp", "sf_city_boundary.shp", "SFIntersections_Voronoi_Clipped.shp")

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\SFIntersections_Voronoi_Clipped.shp'>

# Aggregate Collisions within Voronoi Polygons

In [61]:
AN.SummarizeWithin("SFIntersections_Voronoi_Clipped.shp", "collisions.shp", "SFIntersections_Voronoi_Collisions")
DM.AlterField("SFIntersections_Voronoi_Collisions", "Point_Count", "Total_Collisions", "Total_Collisions")
DM.FeatureToPoint("SFIntersections_Voronoi_Collisions", "Intersection_Points_With_Counts.shp", "CENTROID")
AN.SummarizeWithin("SFIntersections_Voronoi_Clipped.shp", "collisions.shp", "SFIntersections_Voronoi_Collisions")

id,value
0,C:\Users\918831919\Documents\ArcGIS\Projects\ExpectedAccidents\ExpectedAccidents.gdb\SFIntersections_Voronoi_Collisions
1,


# Select, Aggregate, and Write Various Collision Types

In [62]:
# Old approach doesn't account for mutually exclusive events
# args_list = [
#    ['n_broadsides', '"type_of_co" = \'Broadside\'', 'broadsides'],
#    ['n_headons', '"type_of_co" = \'Head-On\'', 'headons'],
#    ['n_readends', '"type_of_co" = \'Rear End\'', 'rearends'],
#    ['n_dark', '"lighting" = \'Dark - Street Lights\'', 'darkness'],
#    ['n_wet', '"road_surfa" = \'Wet\'', 'wetness'],
#    ['n_proceeding_straight', '"party1_mov" = \'Proceeding Straight\'', 'proceeding_straight'],
#    ['n_making_left_turn', '"party1_mov" = \'Making Left Turn\'', 'making_left_turn'],
#    ['n_unsafe_speed', '"pcf_viol_c" = \'Unsafe speed for prevailing conditions\'', 'unsafe_speed'],
#    ['n_unsafe_turn_or_lane_change', '"pcf_viol_c" = \'Unsafe turn or lane change prohibited\'', 'unsafe_turn']
# ]

# Lump Extra Levels into 'Other'

In [63]:
collisions = 'collisions3310.shp'
intersections = 'SFIntersections_Voronoi_Clipped.shp'

In [64]:
fct_lump(collisions, 'pcf_viol_c', 5)
fct_lump(collisions, 'type_of_co', 5)
fct_lump(collisions, 'lighting', 3)
fct_lump(collisions, 'road_surfa', 2)
fct_lump(collisions, 'party1_mov', 5)

# Recode Values

In [65]:
with arcpy.da.UpdateCursor(collisions, ['pcf_viol_c']) as cursor:
    for row in cursor:
        if row[0] == 'Red signal - driver or bicyclist responsibilities':
            row[0] = 'Red signal'
        if row[0] == 'Unsafe speed for prevailing conditions':
            row[0] = 'Unsafe speed'
        if row[0] == 'Unsafe turn or lane change prohibited':
            row[0] = 'Unsafe turn or lane change'
        if row[0] == 'Violation of right-of-way - left turn':
            row[0] = 'Violation of right of way left turn'
        cursor.updateRow(row)
del row, cursor

In [66]:
field_vals_list = [
    ['pcf_viol_c', unique_values(collisions, 'pcf_viol_c')],
    ['type_of_co', unique_values(collisions, 'type_of_co')],
    ['lighting', unique_values(collisions, 'lighting')],
    ['road_surfa', unique_values(collisions, 'road_surfa')],
    ['party1_mov', unique_values(collisions, 'party1_mov')]    
]

In [67]:
field_vals_list

[['pcf_viol_c', ['Following too closely prohibited', 'Other', 'Red signal', 'Unsafe speed', 'Unsafe turn or lane change', 'Violation of right of way left turn']], ['type_of_co', ['Broadside', 'Head-On', 'Hit Object', 'Other', 'Rear End', 'Sideswipe']], ['lighting', ['Dark - Street Lights', 'Daylight', 'Dusk - Dawn', 'Other']], ['road_surfa', ['Dry', 'Other', 'Wet']], ['party1_mov', ['Changing Lanes', 'Making Left Turn', 'Making Right Turn', 'Making U Turn', 'Other', 'Proceeding Straight']]]

In [68]:
for arr in field_vals_list:
    nm = arr[0]
    for level in arr[1]:
        fname = f"n_{level}".replace(" ", "_")
        fname = fname.replace("-", "_")
        fname = fname[0:30]
        outnm = f"{nm}_{level}".replace(" ", "_")
        outnm = outnm.replace("-", "_")
        outnm = outnm[0:30]
        sql = f'"{nm}" = \'{level}\''
        # print(sql)
        SelectAggregateWrite(collisions, intersections, sql, fname, outnm)

653 selected.
pcf_viol_c_Following_too_close created.
3635 selected.
pcf_viol_c_Other created.
1117 selected.
pcf_viol_c_Red_signal created.
2072 selected.
pcf_viol_c_Unsafe_speed created.
881 selected.
pcf_viol_c_Unsafe_turn_or_lane created.
725 selected.
pcf_viol_c_Violation_of_right_ created.
3406 selected.
type_of_co_Broadside created.
713 selected.
type_of_co_Head_On created.
457 selected.
type_of_co_Hit_Object created.
878 selected.
type_of_co_Other created.
2106 selected.
type_of_co_Rear_End created.
1523 selected.
type_of_co_Sideswipe created.
2802 selected.
lighting_Dark___Street_Lights created.
5777 selected.
lighting_Daylight created.
329 selected.
lighting_Dusk___Dawn created.
175 selected.
lighting_Other created.
7989 selected.
road_surfa_Dry created.
217 selected.
road_surfa_Other created.
877 selected.
road_surfa_Wet created.
426 selected.
party1_mov_Changing_Lanes created.
1360 selected.
party1_mov_Making_Left_Turn created.
368 selected.
party1_mov_Making_Right_Turn cre

# Merge Together (bind rows)

In [69]:
to_merge = []
for arr in field_vals_list:
    nm = arr[0]
    for level in arr[1]:
        outnm = f"{nm}_{level}".replace(" ", "_")
        outnm = outnm.replace("-", "_")
        outnm = outnm[0:30]
        to_merge.append(outnm)
to_merge.append('SFIntersections_Voronoi_Collisions')
DM.Merge(to_merge, 'merged_collisions.shp')

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\merged_collisions.shp'>